# 🤖 Emotion Detection (Your CNN + Human Model)
Test your own trained CNN vs a lightweight model from human-models-main.

In [ ]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import img_to_array
import json


In [ ]:

# === CONFIGURATION ===
image_size = (48, 48)
image_folder = "images"
EMOTIONS = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

# === LOAD KERAS MODEL ===
def load_keras_model(json_path, weights_path):
    with open(json_path, "r") as f:
        model = model_from_json(f.read())
    model.load_weights(weights_path)
    return model

model_keras = load_keras_model("models/emotion_model.json", "models/emotion_model_weights.h5")
print("✅ Keras model loaded")


In [ ]:

# === LOAD HUMAN MODEL (JSON + BIN) ===
def load_human_model(json_path):
    with open(json_path, "r") as f:
        model_json = json.load(f)
    return model_json

# Only loading the json structure here
human_model_json = load_human_model("/mnt/data/human-models-main/models/emotion.json")
print("✅ Human model (metadata) loaded")


In [ ]:

# === PREDICT USING KERAS MODEL ===
def predict_emotions_keras(folder):
    for img_file in os.listdir(folder):
        if not img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        path = os.path.join(folder, img_file)
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, image_size)

        face_img = resized.astype("float32") / 255.0
        face_img = np.expand_dims(img_to_array(face_img), axis=0)

        preds = model_keras.predict(face_img)[0]
        label = EMOTIONS[np.argmax(preds)]
        confidence = np.max(preds)

        print(f"[Keras] {img_file} → {label} ({confidence*100:.2f}%)")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Keras: {label} ({confidence*100:.2f}%)")
        plt.axis('off')
        plt.show()


In [ ]:

# === PREDICT USING HUMAN MODEL (Placeholder for now) ===
def predict_emotions_webmodel(folder):
    print("⚠️ Web/ONNX model placeholder — add ONNX or JS inference backend if needed.")
    print("Model info:", human_model_json.get("name", "N/A"), "| format:", human_model_json.get("format", "N/A"))
